In [1]:
!pip install transformers
!pip install sentencepiece
!pip install segtok
!pip install vaderSentiment
!pip install nltk
!pip install huggingface_hub

In [2]:
import os
import sys
from pathlib import Path

import json
import pandas as pd
import random

import torch
from segtok import tokenizer

import tqdm

from multiprocessing import Pool

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import tokenize

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import os
import copy
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ROOT_FOLDER = Path("/content/drive/My Drive/cs182_final_project/cs182-nlp")
DATA_FOLDER = ROOT_FOLDER / "dataset"
TORCH_CHECKPOINT_MODEL = ROOT_FOLDER / "models" / "training_checkpoint_oscar_vader.pt"

input("Please check to make sure the above checkpoint directory is yours (Hit any key)")

Please check to make sure the above checkpoint directory is yours (Hit any key)


''

In [5]:
sys.path.append(str(ROOT_FOLDER))

import data
import models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
import json
import pickle
import sys

import nltk
import torch
import tqdm

import data
import models
from models import LanguageModel

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

MAX_LEN = 128
MAX_LEN_VADER = 40
BATCH_SIZE = 64
EPOCHS = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_params = torch.load(
    TORCH_CHECKPOINT_MODEL, map_location=device
)

model = models.LanguageModel(
    vocab_size=MAX_LEN,
    rnn_size=256,
    vader_size=MAX_LEN_VADER,
    use_vader=True,
    use_bert=False,
    use_cnn=True,
)
model.load_state_dict(model_params["model_state_dict"])
model = model.to(device)
model.eval()

analyzer = SentimentIntensityAnalyzer()

def predict_stars(text):
    """
    text - a SINGLE texts
    """
    # This is where you call your model to get the number of stars output
    encodings = model.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        return_token_type_ids=False,
        return_attention_mask=False,
        truncation=True,
        pad_to_max_length=False,
    )
    text_encoding = encodings.get("input_ids", [])
    vectorized, _ = data.pad_sequence(text_encoding, 0, MAX_LEN)

    sentence_list = nltk.tokenize.sent_tokenize(
        text
    )  # Text is one at a time anyway here
    review_sentiment_sentence = []
    for sentence in sentence_list:
        vs = analyzer.polarity_scores(sentence)
        review_sentiment_sentence.append(vs["compound"])
    vadar_sentiments, _ = data.pad_sequence(review_sentiment_sentence, 0, MAX_LEN_VADER)

    # Place the data as a batch, even if there is only 1
    vectorized = data.batch_to_torch_long([vectorized])
    vadar_sentiments = data.batch_to_torch_float([vadar_sentiments])

    p = model.predict(vectorized, vadar_sentiments)
    return float(p[0].item())

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


In [ ]:
if len(sys.argv) > 1:
    validation_file = "/content/yelp_review_training_dataset.jsonl"
    with open("output.jsonl", "w") as fw:
        with open(validation_file, "r") as fr:
            i = 0
            for line in fr:
                if i % 10 == 0:
                  print(i)
                if i > 500:
                  print("END")
                  break
                review = json.loads(line)
                fw.write(
                    json.dumps(
                        {
                            "review_id": review["review_id"],
                            "predicted_stars": predict_stars(review["text"]),
                        }
                    )
                    + "\n"
                )
                i += 1
    print("Output prediction file written")
else:
    print("No validation file given")

0
10
20
30
40
50
60
70
80
90
100
